# Activity 4.1 - Cleaning Walmart Data the OpenRefine Way

In this activity, you will practice what you learned in Lecture 4.5 by cleaning up a data set containing information on various Walmart locations.

In [24]:
import pandas as pd
from dfply import *

#### Initial Tasks

1. Try to read in the `./data/Walmart_United_States_&_Canada.csv` file and verify that you get an encoding error.  This means that the [character encoding](https://en.wikipedia.org/wiki/Character_encoding) isn't the default of `utf-8`.  The easiest way to fix this is to open and save the file in Visual Studio Code.

In [25]:
walmart = pd.read_csv('data/Walmart_United_States_&_Canada.csv')

2. Read in the data to verify that the encoding is fixed, but that there are two more problems.  What are they?

In [26]:
walmart = pd.read_csv('data/Walmart_United_States_&_Canada.csv')

In [27]:
walmart.head()

,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295"
0,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
1,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
2,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
3,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,..."
4,-113.911590,51.040090,"Walmart Supercentre; #1136,","255 E Hills Blvd SE,Calgary ,AB T2A 4X7,(403) ..."


The columns do not have headers. The first value has been filled into the header. The store type and identifier are in one column separated by a semicolon. There is a random comma after the store number. The address, phone number, and other information is all stored in one column.

3. Take another look at the file in VS Code and determine solutions to the two/three issues, then read in the data correctly by passing `pd.read_csv` the correct defaults for this data. **Note.** Leave the `"` in place for now, as they serve an important role here!

In [28]:
# help(pd.read_csv) # This might help!

We need to add column headers here.

In [41]:
headers = ['lat', 'long', 'store', 'address', 'province/state', 'phone']

walmart_headers = pd.read_csv('data/Walmart_United_States_&_Canada.csv', names = headers, sep = ',')

In [42]:
walmart_headers.head(2)

,lat,long,store,address,province/state,phone
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",NaN,NaN
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",NaN,NaN


## Cleaning up the store information.

As hinted at above, the presence of the `"` meant the two of the columns--one containing the store type/number and the other contain the address/phone number--are combined together.  This was done because some of these entries have a different number of variables.  For example, the store type/number column sometimes occasionally `Gas`.

In this part of the activity, you should apply the iterative OpenRefine approach to separate the information in the store column.

**Warning!** There is one entry that doesn't follow the same pattern as the rest.  You won't find this entry unless you carefully define/fix/eliminate patterns.

In [30]:
from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter

# Your code here.

In [78]:
(walmart_headers
    >> select(X.store)
    >> filter_by(~text_filter(X.store, 'Gas/Diesel', regex = True))
    >> filter_by(~text_filter(X.store, 'Gas,$', regex = True))
)

,store
0,"Walmart Supercentre; #1050,"
1,"Walmart Supercentre; #3658,"
2,"Walmart Supercentre; #3013,"
4,"Walmart; #1144,"
5,"Walmart Supercentre; #1136,"
...,...
6810,"Walmart Supercenter; #1412,"
6812,"Walmart Supercenter; #1457,"
6813,"Walmart Supercenter; #1461,"
6814,"Walmart Supercenter; #1508,"


In [124]:
(walmart_headers
    >> select(X.store)
    >> filter_by(text_filter(X.store, r'Walmart( Supercentre| Supercenter)?; #\d{4},', regex = True))
    # >> filter_by(~text_filter(X.store, r'Murphy: USA;', regex = True))
    # >> filter_by(~text_filter(X.store, r'Wm Nbrhd Mkt;', regex = True))
    # >> filter_by(~text_filter(X.store, r"Sam's Club;", regex = True))
    # >> filter_by(~text_filter(X.store, r'Wm Pharmacy/Clinic;', regex = True))
)

/home/wz8878wh/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,store
0,"Walmart Supercentre; #1050,"
1,"Walmart Supercentre; #3658,"
2,"Walmart Supercentre; #3013,"
3,"Walmart Supercentre; #3009,Gas,"
4,"Walmart; #1144,"
...,...
6811,"Walmart Supercenter; #4471,Gas,"
6812,"Walmart Supercenter; #1457,"
6813,"Walmart Supercenter; #1461,"
6814,"Walmart Supercenter; #1508,"


In [118]:
(walmart_headers
    >> select(X.store)
    >> mutate(fuel_type = case_when((text_filter(X.store, r'Gas/Diesel'), 'Gas/Diesel'),
                                    (text_filter(X.store, r'Gas,$', regex = True), 'Gas'),
                                    (True, 'None')))
    >> mutate(store_type = case_when((text_filter(X.store, r'Walmart( Supercentre| Supercenter)?;', regex = True), X.store.str.split(';').str.get(0)),
                                     (text_filter(X.store, r'Murphy: USA', regex = True), X.store.str.split(';').str.get(0)),
                                     (text_filter(X.store, r'Wm Nbrhd Mkt;', regex = True), X.store.str.split(';').str.get(0)),
                                     (text_filter(X.store, r"Sam's Club;", regex = True), X.store.str.split(';').str.get(0)),
                                     (text_filter(X.store, r'Wm Pharmacy/Clinic;', regex = True), X.store.str.split(';').str.get(0))))
)
                                     
    # >> mutate(store_number = case_when((text_filter(X.store, 'Walmart( Supercentre| Supercenter)?;', X.store.str.extract('(/d/d/d/d)')))))


/home/wz8878wh/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,store,fuel_type,store_type
0,"Walmart Supercentre; #1050,",None,Walmart Supercentre
1,"Walmart Supercentre; #3658,",None,Walmart Supercentre
2,"Walmart Supercentre; #3013,",None,Walmart Supercentre
3,"Walmart Supercentre; #3009,Gas,",Gas,Walmart Supercentre
4,"Walmart; #1144,",None,Walmart
...,...,...,...
6811,"Walmart Supercenter; #4471,Gas,",Gas,Walmart Supercenter
6812,"Walmart Supercenter; #1457,",None,Walmart Supercenter
6813,"Walmart Supercenter; #1461,",None,Walmart Supercenter
6814,"Walmart Supercenter; #1508,",None,Walmart Supercenter


## Preview of Coming Attractions

In this module's homework assignment, you will continue to clean up this data set.